# Real Lab Workflow with Folio

This notebook demonstrates a realistic lab workflow where:
- Observations are entered manually (as you would in a real experiment)
- Notes document experimental details and observations
- Data is exported to pandas for analysis

**Scenario**: Optimizing a Suzuki coupling reaction by varying catalyst loading and temperature.

## Setup

In [ ]:
import tempfile
import pandas as pd

from folio.api import Folio
from folio.core.config import TargetConfig
from folio.core.schema import InputSpec, OutputSpec

In [ ]:
db_path = tempfile.mktemp(suffix=".db")
folio = Folio(db_path=db_path)
print(f"Database: {db_path}")

## Define the Experiment

Suzuki coupling optimization:
- **Inputs**: Catalyst loading (mol%), Temperature (°C)
- **Outputs**: Yield (%), Purity (%)
- **Goal**: Maximize yield

In [ ]:
folio.create_project(
    name="suzuki_coupling",
    inputs=[
        InputSpec("catalyst_loading", "continuous", bounds=(0.5, 5.0), units="mol%"),
        InputSpec("temperature", "continuous", bounds=(60.0, 120.0), units="°C"),
    ],
    outputs=[
        OutputSpec("yield", units="%"),
        OutputSpec("purity", units="%"),
    ],
    target_configs=[TargetConfig(objective="yield", objective_mode="maximize")],
)
print("Project 'suzuki_coupling' created!")

## Enter Initial Observations

These are results from preliminary experiments. Each observation includes notes
documenting relevant experimental details.

In [ ]:
# Experiment 1: Low catalyst, low temperature baseline
folio.add_observation(
    project_name="suzuki_coupling",
    inputs={"catalyst_loading": 1.0, "temperature": 80.0},
    outputs={"yield": 45.2, "purity": 98.1},
    tag="screening",
    notes="Baseline conditions. Reaction incomplete after 24h. TLC shows starting material.",
)
print("Observation 1 recorded")

In [ ]:
# Experiment 2: Higher temperature
folio.add_observation(
    project_name="suzuki_coupling",
    inputs={"catalyst_loading": 1.0, "temperature": 100.0},
    outputs={"yield": 62.8, "purity": 96.5},
    tag="screening",
    notes="Increased temperature helped. Some darkening observed. Reaction complete in 12h.",
)
print("Observation 2 recorded")

In [ ]:
# Experiment 3: Higher catalyst loading
folio.add_observation(
    project_name="suzuki_coupling",
    inputs={"catalyst_loading": 2.5, "temperature": 80.0},
    outputs={"yield": 71.3, "purity": 97.8},
    tag="screening",
    notes="Better conversion with more catalyst. No color change. 18h reaction time.",
)
print("Observation 3 recorded")

In [ ]:
# Experiment 4: Both higher
folio.add_observation(
    project_name="suzuki_coupling",
    inputs={"catalyst_loading": 2.5, "temperature": 100.0},
    outputs={"yield": 83.5, "purity": 95.2},
    tag="screening",
    notes="Best yield so far but purity dropped. Dark brown solution. Consider shorter time.",
)
print("Observation 4 recorded")

In [ ]:
# Experiment 5: Edge of parameter space
folio.add_observation(
    project_name="suzuki_coupling",
    inputs={"catalyst_loading": 4.0, "temperature": 90.0},
    outputs={"yield": 78.9, "purity": 94.1},
    tag="screening",
    notes="High catalyst loading. Significant Pd black formation. May be wasteful.",
)
print("Observation 5 recorded")

## Get a Suggestion from Folio

Now let's ask Folio for the next experiment to try.

In [ ]:
suggestion = folio.suggest("suzuki_coupling")[0]
print("Folio suggests:")
print(f"  Catalyst loading: {suggestion['catalyst_loading']:.2f} mol%")
print(f"  Temperature: {suggestion['temperature']:.1f} °C")

## Run the Suggested Experiment

After running the experiment in the lab, record the results:

In [ ]:
# Record the result of the suggested experiment
# (In real use, you would fill in actual measured values)
folio.add_observation(
    project_name="suzuki_coupling",
    inputs=suggestion,
    outputs={"yield": 87.2, "purity": 96.8},
    tag="optimization",
    notes="BO-suggested conditions. Clean reaction, good conversion. Optimal color (pale yellow).",
)
print("Observation 6 recorded (BO-suggested)")

## Continue the Optimization Loop

Get another suggestion and record results:

In [ ]:
suggestion2 = folio.suggest("suzuki_coupling")[0]
print("Next suggestion:")
print(f"  Catalyst loading: {suggestion2['catalyst_loading']:.2f} mol%")
print(f"  Temperature: {suggestion2['temperature']:.1f} °C")

In [ ]:
folio.add_observation(
    project_name="suzuki_coupling",
    inputs=suggestion2,
    outputs={"yield": 89.1, "purity": 97.2},
    tag="optimization",
    notes="Excellent result. Consider this as optimized conditions for scale-up.",
)
print("Observation 7 recorded")

## Export to Pandas DataFrame

Export all observations to a pandas DataFrame for analysis, plotting, or saving to CSV.

In [ ]:
def observations_to_dataframe(observations: list) -> pd.DataFrame:
    """Convert Folio observations to a pandas DataFrame."""
    records = []
    for obs in observations:
        record = {
            "id": obs.id,
            "timestamp": obs.timestamp,
            "tag": obs.tag,
            "notes": obs.notes,
        }
        # Add inputs
        for key, value in obs.inputs.items():
            record[f"input_{key}"] = value
        # Add outputs
        for key, value in obs.outputs.items():
            record[f"output_{key}"] = value
        records.append(record)
    return pd.DataFrame(records)

In [ ]:
# Get all observations and convert to DataFrame
observations = folio.get_observations("suzuki_coupling")
df = observations_to_dataframe(observations)

print(f"Total observations: {len(df)}")
df

## Filter by Tag

View only the optimization experiments:

In [ ]:
opt_observations = folio.get_observations("suzuki_coupling", tag="optimization")
df_opt = observations_to_dataframe(opt_observations)

print("Optimization experiments only:")
df_opt[["input_catalyst_loading", "input_temperature", "output_yield", "output_purity", "notes"]]

## View Notes

Review experimental notes for all runs:

In [ ]:
print("Experimental Notes")
print("=" * 60)
for obs in observations:
    print(f"\n[{obs.tag}] Cat: {obs.inputs['catalyst_loading']} mol%, T: {obs.inputs['temperature']}°C")
    print(f"  Yield: {obs.outputs['yield']}%, Purity: {obs.outputs['purity']}%")
    print(f"  Notes: {obs.notes}")

## Export to CSV

Save the data for external analysis or archival:

In [ ]:
# Uncomment to save:
# df.to_csv("suzuki_coupling_results.csv", index=False)
# print("Saved to suzuki_coupling_results.csv")

print("CSV export ready (uncomment above to save)")

## Summary

This demo showed a realistic lab workflow:

1. **Create a project** with meaningful input/output names and units
2. **Record observations manually** with detailed notes
3. **Use tags** to categorize experiments (screening vs optimization)
4. **Get BO suggestions** for the next experiment
5. **Export to pandas** for analysis and reporting

The notes field is valuable for:
- Recording qualitative observations (color changes, precipitates)
- Documenting deviations from protocol
- Capturing insights for future reference

In [ ]:
# Cleanup
folio.delete_project("suzuki_coupling")
print("Demo complete!")